In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures  

In [2]:
df=pd.read_excel('clean_data.xlsx')
df.head()

,Date,Time,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,2004-03-10,18:00:00,2.6,1360.00,11.881723,1045.50,166.0,1056.25,113.0,1692.00,1267.50,13.60,48.875001,0.757754
1,2004-03-10,19:00:00,2.0,1292.25,9.397165,954.75,103.0,1173.75,92.0,1558.75,972.25,13.30,47.700000,0.725487
2,2004-03-10,20:00:00,2.2,1402.00,8.997817,939.25,131.0,1140.00,114.0,1554.50,1074.00,11.90,53.975000,0.750239
3,2004-03-10,21:00:00,2.2,1375.50,9.228796,948.25,172.0,1092.00,122.0,1583.75,1203.25,11.00,60.000000,0.786713
4,2004-03-10,22:00:00,1.6,1272.25,6.518224,835.50,131.0,1205.00,116.0,1490.00,1110.00,11.15,59.575001,0.788794


In [3]:
seasons={}

In [4]:
spring = df.loc[df['Date']<'2004-06-01']
spring = spring.reset_index(drop=True)
seasons['spring']=spring

In [5]:
summer = df.loc[(df['Date']<'2004-09-01') & (df['Date']>='2004-06-01')]
summer = summer.reset_index(drop=True)
seasons['summer']=summer

In [6]:
autumn = df.loc[(df['Date']<'2004-12-01') & (df['Date']>='2004-09-01')]
autumn = autumn.reset_index(drop=True)
seasons['autumn']=autumn

In [7]:
winter = df.loc[(df['Date']<'2005-03-01') & (df['Date']>='2004-12-01')]
winter = winter.reset_index(drop=True)
seasons['winter']=winter

In [8]:
summer.to_excel('summer.xlsx', index=False)
spring.to_excel('spring.xlsx', index=False)
autumn.to_excel('autumn.xlsx', index=False)
winter.to_excel('winter.xlsx', index=False)

In [9]:
cols = df.columns
cols

Index(['Date', 'Time', 'CO(GT)', 'PT08.S1(CO)', 'C6H6(GT)', 'PT08.S2(NMHC)',
       'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)', 'PT08.S5(O3)',
       'T', 'RH', 'AH'],
      dtype='object')

In [10]:
for season in seasons:
    for col in cols[2:-3]:
        X=seasons[season]['RH'].values.reshape(-1,1)
        y=seasons[season][col].values.reshape(-1,1)

        reg = LinearRegression().fit(X, y)
        r2 = reg.score(X, y)
        c = reg.coef_
        i = reg.intercept_

        plt.scatter(X, y)
        y_pred = reg.predict(X)

        plt.plot(X, y_pred, color ='orange')
        plt.xlabel('Relative Humidity')
        plt.ylabel(col)
        plt.suptitle(season)
        plt.title(('coef = ',np.round(c,2)[0][0],' intercept = ',np.round(i,2)[0]))
#         plt.show()
        plt.savefig(season+' '+col+'_new'+'.png')
        plt.close()

In [27]:
summer.corr().to_excel('summer_corr.xlsx')
winter.corr().to_excel('winter_corr.xlsx')
autumn.corr().to_excel('autumn_corr.xlsx')
spring.corr().to_excel('spring_corr.xlsx')

In [43]:
y=spring['RH'].values.reshape(-1,1)
x=[]
for col in spring.columns:
    if col not in ['Date', 'Time', 'RH']:
        x.append(spring[col].values)
x=np.transpose(x)
# x=np.reshape(x,(len(x), len(x[0])))

In [29]:
from sklearn.metrics import mean_absolute_error

In [30]:
reg = LinearRegression().fit(x, y)

In [31]:
reg.score(x,y)

0.9554495768326872

In [32]:
y_pred=reg.predict(x)
mean_absolute_error(y, y_pred)

2.67736954278484

In [33]:
test = pd.read_excel('clean_data.xlsx')

In [34]:
test = test[(test['Date']>='2005-03-01') & (test['Date']<='2005-03-31')]

In [35]:
test=test.reset_index(drop=True)

In [36]:
y_test=test['RH'].values.reshape(-1,1)
x_test=[]
for col in test.columns:
    if col not in ['Date', 'Time', 'RH']:
        x_test.append(test[col].values)
x_test=np.transpose(x_test)
x_test=np.reshape(x_test,(len(x_test), len(x_test[0])))

In [37]:
reg.score(x_test,y_test)

0.7494622697620106

In [38]:
from keras.layers import Flatten, Dense, BatchNormalization
from keras import Input, layers
from keras.models import Model

In [39]:
n_units=50
input_tensor = Input(shape=(len(x[0]),))
dense = Dense(n_units, activation='relu')(input_tensor)

for i in range(5):
    dense = Dense(n_units, activation='relu')(dense)
    
dense=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(dense)
output_tensor = Dense(1, activation='relu')(dense)

model = Model(input_tensor, output_tensor)

model.compile(optimizer='sgd', loss='mse')

history = model.fit(x, y, epochs=500, batch_size=32)

Epoch 1/500
62/62 [==============================] - 0s 1ms/step - loss: 408.5066
Epoch 2/500
62/62 [==============================] - 0s 1ms/step - loss: 291.8640
Epoch 3/500
62/62 [==============================] - 0s 2ms/step - loss: 228.7855
Epoch 4/500
62/62 [==============================] - 0s 2ms/step - loss: 233.4787
Epoch 5/500
62/62 [==============================] - 0s 2ms/step - loss: 170.5002
Epoch 6/500
62/62 [==============================] - 0s 2ms/step - loss: 150.5244
Epoch 7/500
62/62 [==============================] - 0s 1ms/step - loss: 169.4309
Epoch 8/500
62/62 [==============================] - 0s 2ms/step - loss: 136.9797
Epoch 9/500
62/62 [==============================] - 0s 1ms/step - loss: 159.2082
Epoch 10/500
62/62 [==============================] - 0s 2ms/step - loss: 148.5874
Epoch 11/500
62/62 [==============================] - 0s 2ms/step - loss: 144.5597
Epoch 12/500
62/62 [==============================] - 0s 2ms/step - loss: 142.9863
Epoch 13/500


62/62 [==============================] - 0s 2ms/step - loss: 118.3879
Epoch 101/500
62/62 [==============================] - 0s 1ms/step - loss: 117.5030
Epoch 102/500
62/62 [==============================] - 0s 2ms/step - loss: 117.4586
Epoch 103/500
62/62 [==============================] - 0s 2ms/step - loss: 122.8290
Epoch 104/500
62/62 [==============================] - 0s 2ms/step - loss: 113.2068
Epoch 105/500
62/62 [==============================] - 0s 2ms/step - loss: 111.0012
Epoch 106/500
62/62 [==============================] - 0s 3ms/step - loss: 115.0596
Epoch 107/500
62/62 [==============================] - 0s 2ms/step - loss: 114.8983
Epoch 108/500
62/62 [==============================] - 0s 2ms/step - loss: 118.7806
Epoch 109/500
62/62 [==============================] - 0s 1ms/step - loss: 114.0511
Epoch 110/500
62/62 [==============================] - 0s 2ms/step - loss: 119.1265
Epoch 111/500
62/62 [==============================] - 0s 2ms/step - loss: 110.7040
Epoch 

62/62 [==============================] - 0s 2ms/step - loss: 103.8788
Epoch 199/500
62/62 [==============================] - 0s 2ms/step - loss: 83.8351
Epoch 200/500
62/62 [==============================] - 0s 2ms/step - loss: 101.9034
Epoch 201/500
62/62 [==============================] - 0s 1ms/step - loss: 89.1229
Epoch 202/500
62/62 [==============================] - 0s 1ms/step - loss: 99.3433
Epoch 203/500
62/62 [==============================] - 0s 2ms/step - loss: 93.3958
Epoch 204/500
62/62 [==============================] - 0s 2ms/step - loss: 101.3146
Epoch 205/500
62/62 [==============================] - 0s 2ms/step - loss: 95.5411
Epoch 206/500
62/62 [==============================] - 0s 1ms/step - loss: 88.2596
Epoch 207/500
62/62 [==============================] - 0s 1ms/step - loss: 98.3473
Epoch 208/500
62/62 [==============================] - 0s 1ms/step - loss: 88.9644
Epoch 209/500
62/62 [==============================] - 0s 1ms/step - loss: 92.9255
Epoch 210/500
6

62/62 [==============================] - 0s 1ms/step - loss: 80.5742
Epoch 297/500
62/62 [==============================] - 0s 1ms/step - loss: 82.4497
Epoch 298/500
62/62 [==============================] - 0s 2ms/step - loss: 78.4698
Epoch 299/500
62/62 [==============================] - 0s 1ms/step - loss: 63.7602
Epoch 300/500
62/62 [==============================] - 0s 2ms/step - loss: 83.5708
Epoch 301/500
62/62 [==============================] - 0s 2ms/step - loss: 91.5484
Epoch 302/500
62/62 [==============================] - 0s 2ms/step - loss: 76.2954
Epoch 303/500
62/62 [==============================] - 0s 1ms/step - loss: 73.3834
Epoch 304/500
62/62 [==============================] - 0s 1ms/step - loss: 70.6927
Epoch 305/500
62/62 [==============================] - 0s 1ms/step - loss: 79.9470
Epoch 306/500
62/62 [==============================] - 0s 1ms/step - loss: 82.9048
Epoch 307/500
62/62 [==============================] - 0s 1ms/step - loss: 73.3857
Epoch 308/500
62/6

62/62 [==============================] - 0s 2ms/step - loss: 84.2116
Epoch 395/500
62/62 [==============================] - 0s 2ms/step - loss: 65.8471
Epoch 396/500
62/62 [==============================] - 0s 2ms/step - loss: 73.5200
Epoch 397/500
62/62 [==============================] - 0s 1ms/step - loss: 64.0477
Epoch 398/500
62/62 [==============================] - 0s 1ms/step - loss: 67.4240
Epoch 399/500
62/62 [==============================] - 0s 1ms/step - loss: 70.4472
Epoch 400/500
62/62 [==============================] - 0s 1ms/step - loss: 58.2485
Epoch 401/500
62/62 [==============================] - 0s 1ms/step - loss: 57.0280
Epoch 402/500
62/62 [==============================] - 0s 2ms/step - loss: 63.1844
Epoch 403/500
62/62 [==============================] - 0s 2ms/step - loss: 65.8821
Epoch 404/500
62/62 [==============================] - 0s 2ms/step - loss: 68.4870
Epoch 405/500
62/62 [==============================] - 0s 1ms/step - loss: 65.5245
Epoch 406/500
62/6

62/62 [==============================] - 0s 1ms/step - loss: 59.3652
Epoch 493/500
62/62 [==============================] - 0s 1ms/step - loss: 58.0586
Epoch 494/500
62/62 [==============================] - 0s 1ms/step - loss: 54.6830
Epoch 495/500
62/62 [==============================] - 0s 2ms/step - loss: 55.5536
Epoch 496/500
62/62 [==============================] - 0s 2ms/step - loss: 68.1771
Epoch 497/500
62/62 [==============================] - 0s 2ms/step - loss: 66.7312
Epoch 498/500
62/62 [==============================] - 0s 1ms/step - loss: 55.6392
Epoch 499/500
62/62 [==============================] - 0s 2ms/step - loss: 60.4156
Epoch 500/500
62/62 [==============================] - 0s 1ms/step - loss: 51.6568


In [40]:
model.evaluate(x_test, y_test)

24/24 [==============================] - 0s 1ms/step - loss: 707.3721


707.3720703125

In [41]:
for season in seasons:
    for col in cols[2:-3]:
        X=seasons[season]['RH'].values.reshape(-1,1)
        y=seasons[season][col].values.reshape(-1,1)
        poly_regs= PolynomialFeatures(degree= 4)  
        x_poly= poly_regs.fit_transform(X)  
        lin_reg_2 =LinearRegression()  
        lin_reg_2.fit(x_poly, y)
        r2 = lin_reg_2.score(x_poly, y)
        c = lin_reg_2.coef_
        i = lin_reg_2.intercept_

        plt.scatter(X, y)
        y_pred = lin_reg_2.predict(x_poly)

        plt.plot(X, y_pred, color ='orange')
        plt.xlabel('Relative Humidity')
        plt.ylabel(col)
        plt.suptitle(season)
        plt.title(('coef = ',np.round(c,2)[0][0],' intercept = ',np.round(i,2)[0]))
#         plt.show()
        plt.savefig(season+' '+col+'_new2'+'.png')
        plt.close()

In [44]:
poly_regs= PolynomialFeatures(degree= 4)  
x_poly= poly_regs.fit_transform(x)  
lin_reg_2 =LinearRegression()  
lin_reg_2.fit(x_poly, y)
lin_reg_2.score(x_poly,y)

0.9999999564144426

In [45]:
y_pred=lin_reg_2.predict(x_poly)
mean_absolute_error(y, y_pred)

0.0025654641846171